<a href="https://colab.research.google.com/github/elbernaderen/final-project-cs50/blob/master/final_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Instalación de python, librerias con mapas y flask en el servidor

In [ ]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express
# Install flask-ngrok to run flask on colab
!pip install flask-ngrok
# Installjavascript on colab
!npm install -g --unsafe-perm ijavascript zeromq
!ijsinstall --install=global
! pip install Flask-Session
! pip install Flask

# Base de datos

In [ ]:
import sqlite3

conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db')
print("Opened database successfully");

conn.execute('''
CREATE TABLE IF NOT EXISTS claims(id_claim INTEGER,
                      name TEXT,
                      address TEXT, 
                      latitude REAL, 
                      longitude REAL, 
                      date TEXT,
                      reason TEXT,
                      PRIMARY KEY(id_claim));''')

conn.commit()

print("Table created successfully");

conn.close()

Opened database successfully
Table created successfully


In [ ]:
import sqlite3

conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db')
print("Opened database successfully");

conn.execute('''
CREATE TABLE IF NOT EXISTS users (id INTEGER,
                              username TEXT,
                              hash TEXT, 
                              PRIMARY KEY(id));''')
conn.commit()

print("Table created successfully");

conn.close()

Opened database successfully
Table created successfully


# aplicación

https://stackoverflow.com/questions/52638926/geo-coding-for-intersection-street-using-python from here i have taken the idea for searching latitude and longitude from googlemaps

In [ ]:
import os
import sys
sys.path.append('/content/drive/MyDrive/final')
import urllib.parse

from flask_ngrok import run_with_ngrok
from flask_session import Session
from datetime import datetime
from tempfile import mkdtemp
from flask import Flask, flash, redirect, render_template, request, session, make_response
from geopandas.tools import geocode
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import requests
import sqlite3
import re
from werkzeug.exceptions import default_exceptions, HTTPException, InternalServerError
from werkzeug.security import check_password_hash, generate_password_hash
from helpers import login_required
from pytz import timezone
import random
import json

app = Flask(__name__,template_folder='/content/drive/MyDrive/final/template')
run_with_ngrok(app)
app.config["TEMPLATES_AUTO_RELOAD"] = True

# Ensure responses aren't cached
@app.after_request
def after_request(response):
    response.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    response.headers["Expires"] = 0
    response.headers["Pragma"] = "no-cache"
    return response

app.config["SESSION_FILE_DIR"] = mkdtemp()
app.config["SESSION_PERMANENT"] = False
app.config["SESSION_TYPE"] = "filesystem"
Session(app)

lat = -31.4186703
long = -64.1903681


@app.route("/", methods=["GET", "POST"])
def index():
        conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
        m_1 = folium.Map(location = (lat,long), zoom_start = 12)
        m_1.save('/content/drive/MyDrive/final/template/map.html')
        if request.method == "POST":
          name = request.form.get("name")
          time = datetime.now(timezone("America/Argentina/Cordoba"))
          direction = request.form.get("direction")
          reason =  request.form.get("reason")
          gps1 = loc(direction)
          gps1[0] = float(gps1[0])
          gps1[1] = float(gps1[1])
          conn.execute("INSERT INTO claims (name, address, latitude, longitude, date, reason) VALUES(?, ?, ?, ?, ?, ?)", [name, direction, gps1[0], gps1[1], time, reason])
          conn.row_factory = sqlite3.Row
          cur = conn.cursor()
          cur.execute(''' SELECT id_claim FROM claims WHERE name = ? AND date = ? ;''', [name, time])
          r = cur.fetchone()
          conn.commit()
          return render_template("claimed.html",id_claim = r["id_claim"], name = name)
        else:
          return render_template("index.html")


@app.route("/claims", methods=["GET", "POST"])
@login_required
def claims():
    conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
    m_1 = folium.Map(location = (lat,long), zoom_start = 12)
    cursor = conn.execute(''' SELECT latitude, longitude, reason FROM claims;''')
    
    for row in cursor:
          if row[2] == "Sewer overflow":
            folium.Marker(location=[row[0],row[1]],popup = row[2],icon=folium.Icon(color='blue', prefix='fa')).add_to(m_1)
          else:
            folium.Marker(location=[row[0],row[1]],popup = row[2],icon=folium.Icon(color='red', prefix='fa')).add_to(m_1)        
    conn.close()
    m_1.save('/content/drive/MyDrive/final/template/map.html')
    return render_template('claims.html')





@app.route("/register", methods=["GET", "POST"])
def register():
    """Register user"""
    if request.method == "POST":
        password = request.form.get("password")
        phash = generate_password_hash(password)
        conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
        conn.execute("INSERT INTO users (username, hash) VALUES(?, ?)", [request.form.get("name"), phash])
        rows = conn.execute("""SELECT id FROM users WHERE username = ?""", [request.form.get("name")])
        row = rows.fetchone()
        session["user_id"] = row[0]
        conn.commit()
        # Redirect user to home page
        return redirect("/claims")
    else:
        return render_template("register.html")

@app.route("/login", methods=["GET", "POST"])
def login():
    """Log user in"""

    # Forget any user_id
    session.clear()

    # User reached route via POST (as by submitting a form via POST)
    if request.method == "POST":
        conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
        # Query database for username
        conn.row_factory = sqlite3.Row
        cur = conn.cursor()
        cur.execute("""SELECT * FROM users WHERE username = ?""", [request.form.get("name")])
        r = cur.fetchone()
        conn.commit()      
        # Ensure username exists and password is correct
        if r["username"] == request.form.get("name") and check_password_hash(r["hash"], request.form.get("password")):
          # Remember which user has logged in
          session["user_id"] = r["id"]
          # Redirect user to home page
          return redirect("/claims")
        else:
          return render_template("login.html")
        

    # User reached route via GET (as by clicking a link or via redirect)
    else:
        return render_template("login.html")



@app.route("/logout")
def logout():
    """Log user out"""

    # Forget any user_id
    session.clear()

    # Redirect user to login form
    return redirect("/")

@app.route('/map')
def map():
    return render_template('map.html')


@app.route("/ajax_index", methods=["GET", "POST"])
def ajax_index():
    if request.method == "POST":
      conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
      print("1")
      json_data = json.loads(request.data)
      print(json_data)
      gps1 = loc(json_data["direction"])
      gps1[0] = float(gps1[0])
      gps1[1] = float(gps1[1])
       #   time = datetime.now(timezone("America/Argentina/Cordoba"))
        #  direction = request.form.get("direction")
        #  reason =  request.form.get("reason")
       #   conn.execute("INSERT INTO claims (name, address, latitude, longitude, date, reason) VALUES(?, ?, ?, ?, ?, ?)", [name, direction, gps1[0], gps1[1], time, reason])
       #   conn.row_factory = sqlite3.Row
      #    cur = conn.cursor()
      #    cur.execute(''' SELECT id_claim FROM claims WHERE name = ? AND date = ? ;''', [name, time])
      #    r = cur.fetchone()
      #    conn.commit()
      #    return render_template("claimed.html",id_claim = r["id_claim"], name = name)
      #  else:


      #m_1 = folium.Map(location = (lat,long), zoom_start = 12)
      #cursor = conn.execute(''' SELECT latitude, longitude, reason FROM claims;''')
      m_1 = folium.Map(location = (gps1[0],gps1[1]), zoom_start = 120)
      folium.Marker(location=[gps1[0],gps1[1]]).add_to(m_1)
      

      m_1.save('/content/drive/MyDrive/final/template/map.html')
      print("ok")
      return render_template("get_data.html")    

@app.route("/ajax", methods=["GET", "POST"])
def ajax():
    if request.method == "POST":
      conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
      print("1")
      reason_s = request.form.get("data")
      print(reason_s)
      m_1 = folium.Map(location = (lat,long), zoom_start = 12)
      cursor = conn.execute(''' SELECT latitude, longitude, reason FROM claims;''')
      for row in cursor:
        if reason_s == row[2]:
          folium.Marker(location=[row[0],row[1]],popup = row[2]).add_to(m_1)
        if reason_s == "all":
          if row[2] == "Sewer overflow":
            folium.Marker(location=[row[0],row[1]],popup = row[2],icon=folium.Icon(color='blue', prefix='fa')).add_to(m_1)
          else:
            folium.Marker(location=[row[0],row[1]],popup = row[2],icon=folium.Icon(color='red', prefix='fa')).add_to(m_1)
      conn.close()
      m_1.save('/content/drive/MyDrive/final/template/map.html')
      print("ok")
      return render_template("get_data.html")

def loc(direction):
    try:

      res = requests.get('https://www.google.com/maps/place/'+direction+",+Ciudad+de+Córdoba,+Argentina", cookies={'CONSENT': 'YES+cb.20210328-17-p0.en-GB+FX+{}'.format(random.randint(100, 999))})
      print(res.url)
      print(res.headers)
      return re.findall(r'll=(.*?)" item', res.text)[0].split(',')
    except IndexError:
      
      res = requests.get('https://www.google.com/maps/place/'+direction+",+Ciudad+de+Córdoba,+Argentina", cookies={'CONSENT': 'YES+cb.20210328-17-p0.en-GB+FX+{}'.format(random.randint(100, 999))})
      print(res.url)
      print(res.headers)
      rest = re.findall(r'null,null,-(.*?)]', res.text)[0].split(',')
      rest[0] = float(rest[0]) * -1  
      return rest

app.run()

